In [1]:
import pandas as pd
import re
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import locationtagger

In [2]:
df = pd.read_csv("google_review_29_08_v3.csv")
df.head()

,SVAAS_ID,user_id,firstName,middleName,lastName,email,phone,description,pan,mci,...,numberFollowUp,followUpDuration,insuranceProviderId,isActive,ABDM_Healthcare_Professional_ID,SVAAS_DOCTOR_ID,svaasClinicId,tbl_user_phone_number,name,location
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[['SVAAS_ID', 'firstName', 'lastName', 'name', 'baseCity']]
df.head()

,SVAAS_ID,firstName,lastName,name,baseCity
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


In [4]:
df_not_missing = df.dropna()

In [5]:
df_not_missing.head()

,SVAAS_ID,firstName,lastName,name,baseCity
17,3.0,Divya,Kavuri,Apollo Clinic,Hyderabad
18,4.0,Shivam,P,Shivam hospitals,Hyderabad
19,5.0,Ranjan,Dey,Ranjan skin clinics,Hyderabad
20,6.0,Soumya,Sarkar,Soumya Multi speciality clinic,Hyderabad
21,7.0,Rishanku,Goyal,RKG Eye Center,Hyderabad


In [6]:
df_not_missing.isna().sum()

SVAAS_ID     0
firstName    0
lastName     0
name         0
baseCity     0
dtype: int64

In [7]:
len(df_not_missing)/len(df)

0.9879032258064516

In [8]:
df_not_missing['query'] = df_not_missing['firstName'].apply(lambda x:x.lower()) +"+" + df_not_missing['lastName'].apply(lambda x:x.lower()) \
                          +"+"+ df_not_missing['name'].apply(lambda x:x.lower()) + "+" + df_not_missing["baseCity"].apply(lambda x:x.lower())

/var/folders/z1/0tn2v83n6pzgl2m8lzz1phbc0000gn/T/ipykernel_87780/4047833984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_missing['query'] = df_not_missing['firstName'].apply(lambda x:x.lower()) +"+" + df_not_missing['lastName'].apply(lambda x:x.lower()) \


In [9]:
df_not_missing['query'].values.shape

(1715,)

## For missing data

In [10]:
import preprocess_kgptalkie as ps
import re

def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x




In [11]:
df_not_missing['SVAAS_ID'] = df_not_missing['SVAAS_ID'].astype('int')

/var/folders/z1/0tn2v83n6pzgl2m8lzz1phbc0000gn/T/ipykernel_87780/4195452832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_missing['SVAAS_ID'] = df_not_missing['SVAAS_ID'].astype('int')


In [12]:
df_not_missing['query'] = df_not_missing['query'].apply(lambda x:ps.remove_accented_chars(x))
df_not_missing['query'].head()

/var/folders/z1/0tn2v83n6pzgl2m8lzz1phbc0000gn/T/ipykernel_87780/153722280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_missing['query'] = df_not_missing['query'].apply(lambda x:ps.remove_accented_chars(x))


17                 divya+kavuri+apollo clinic+hyderabad
18                  shivam+p+shivam hospitals+hyderabad
19             ranjan+dey+ranjan skin clinics+hyderabad
20    soumya+sarkar+soumya multi speciality clinic+h...
21              rishanku+goyal+rkg eye center+hyderabad
Name: query, dtype: object

In [13]:
from collections import defaultdict

final_df = defaultdict(list)

In [14]:
def isDigit(char):
    pattern = "^[-+]?[0-9]*\.?[0-9]+(e[-+]?[0-9]+)?$"
    reg = re.compile(pattern)
    if reg.match(char):
        return True
    return False

In [15]:
import requests
from bs4 import BeautifulSoup
import re

def getLinkFromUrl(query):
    link_values = []
    cleaned_links = []
    page = requests.get(query)
    soup = BeautifulSoup(page.content)
    links = soup.findAll("a")
    for link in  soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
        url = re.split(":(?=http)",link["href"].replace("/url?q=",""))
        link_values.append(str(url[0]))
        
    return link_values 

In [16]:
links = getLinkFromUrl(query="https://google.com/search?q=shivam+p+shivam+hospitals+hyderabad")

In [17]:
for l in links:
    if "%" in l:
        link = l.split("%")[0]
        print(link)
    elif "&" in l:
        link = l.split("&")[0]
        print(link)

https://www.justdial.com/Hyderabad/Shivam-Clinic-Near-Hanuman-Temple-Kukatpally/040PXX40-XX40-140310200923-M5T7_BZDET
https://www.justdial.com/Hyderabad/Shivam-Clinic-Obg-Gynecology-and-Orthopedics-Near-Y-Junction-Kukatpally/040PXX40-XX40-150727214202-J1K7_BZDET
https://www.practo.com/hyderabad/clinic/shivam-clinic-kphb
https://www.drmsanjayortho.com/
https://drlogy.com/hospitals/shivam-clinic-23676
https://www.timesmed.com/Hospital/Hyderabad/shivam-clinic-115461
https://www.lybrate.com/hyderabad/clinic/shivam-clinic-kphb
https://ezz.health/hyderabad/hospitals/shivam-clinic-kukatpally
https://theshivamhospital.in/
http://www.shivamhospital.in/
https://support.google.com/websearch
https://accounts.google.com/ServiceLogin


In [18]:
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
import requests
import re

    

query="https://google.com/search?q=shivam+p+shivam+hospitals+hyderabad"

request = urllib.request.Request(query)
request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
raw_response = urllib.request.urlopen(request).read()

html = raw_response.decode("utf-8")

soup = BeautifulSoup(html, 'html.parser')

links = soup.findAll("a")
for link in  soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
    print(re.split(":(?=http)",link["href"].replace("/url?q=","")))


In [19]:
"practo" in "practo.com"

True

In [20]:
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
from googlesearch import search   

url = "https://www.practo.com/hyderabad/doctor/divya-varikallu-general-practitioner"
request = urllib.request.Request(url)

request.add_header('User-Agent', 'your bot 0.1')
raw_response = urllib.request.urlopen(request).read()

html = raw_response.decode("utf-8")

soup = BeautifulSoup(html, 'html.parser')

heading = soup.find("h1")
speciality = soup.find("h2", class_="u-d-inlineblock u-spacer--right-v-thin c-profile__details")
rating = soup.find("span", class_="u-green-text u-bold u-large-font")
votes = soup.find("span", class_="u-smallest-font u-grey_3-text")
address = soup.find("p", class_="c-profile--clinic__address")

print(heading.get_text())
print(speciality.get_text())
# print(rating.get_text())
# print(votes.get_text().strip(")").strip("("))
print(address.get_text())


Dr. Divya Varikallu
General Practitioner
61B, Sri Ram Nagar Colony, Botanical Garden Road, Landmark: Beside Lane of Burfighar Sweet Shop &  State Bank of India Lane, Hyderabad


In [21]:
df_not_missing['query'].apply(lambda x:ps.remove_special_chars(x))

17                      divyakavuriapollo clinichyderabad
18                       shivampshivam hospitalshyderabad
19                  ranjandeyranjan skin clinicshyderabad
20      soumyasarkarsoumya multi speciality clinichyde...
21                   rishankugoyalrkg eye centerhyderabad
                              ...                        
1731    kushangmodshivam children hospital and neonata...
1732              dhavalakhanichildhood hospitalahmedabad
1733    keval patelpateljk dental hospital orthodontic...
1734    jinal patelpateljk dental hospital orthodontic...
1735                      lavanyayasamsai clinichyderabad
Name: query, Length: 1715, dtype: object

In [22]:
set(["doctor"]).issubset(set(list(map(str.lower, link.split("/")))))

False

In [27]:
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
from googlesearch import search   


def extractReviews(df_not_missing):
    queries = df_not_missing['query'].values

    for i in tqdm(range(len(queries))):
        query = queries[i]
        query = "+".join(query.split(" "))
        url = 'https://google.com/search?q=dr+'+query
        

        
        for link in getLinkFromUrl(url):
            if "practo" in link:
                    # link wihtout ampersand
                    if "&" in link:
                        link_w = link.split("&")[0]
                    elif "%" in link:
                        link_w = link.split("%")[0]
                    
                    print("Link searched for: ", link_w)

                    request = urllib.request.Request(link_w)
                    request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

                
                
                    raw_response = urllib.request.urlopen(request).read()
                    # Read the repsonse as a utf-8 string
                    html = raw_response.decode("utf-8")

                    soup = BeautifulSoup(html, 'html.parser')
                    
                    heading = soup.find("h1")

                    # Dr data
                    speciality = soup.find("h2", class_="u-d-inlineblock u-spacer--right-v-thin c-profile__details")
                    rating = soup.find("span", class_="u-green-text u-bold u-large-font")
                    votes = soup.find("span", class_="u-smallest-font u-grey_3-text")
                    address = soup.find("p", class_="c-profile--clinic__address")

                    # clinic data
                    speciality_cl = soup.find("h2", attrs={
                        "data-qa-id":"clinic-speciality"
                    })
                    rating_cl = soup.find("span", class_="common__star-rating__value")
                    votes_cl = soup.find("span", attrs={
                        "data-qa-id":"clinic-votes"
                    })
                    address_cl = soup.find("p", class_="c-profile--clinic__address")
                    
                    final_df['SVAAS_ID'].append(df_not_missing.iloc[i]['SVAAS_ID'])
                    final_df['']


                    if heading is not None:
                        print("Name: ", heading.get_text())
                        final_df['name'].append(heading.get_text())
                    else:
                        print("Heading is not availble")
                        final_df['name'].append("NA")

                    if speciality is not None:
                        print("Speciality: ", speciality.get_text())
                        final_df['speciality'].append(speciality.get_text())
                    elif speciality_cl is not None:
                        print("Speciality: ", speciality_cl.get_text())
                        final_df['speciality'].append(speciality_cl.get_text())
                    else:
                        print("Speciality is not available.")

                    if rating is not None:
                        print("Rating: ", rating.get_text())
                        final_df['rating'].append(rating.get_text())
                    elif rating_cl is not None:
                        print("Rating: ", rating_cl.get_text())
                        final_df['rating'].append(rating_cl.get_text())
                    else:
                        print("Rating not available.")

                    if votes is not None:
                        print("Name: ", votes.get_text())
                        final_df['votes'].append(votes.get_text())
                    elif votes_cl is not None:
                        print("Name: ", votes_cl.get_text())
                        final_df['votes'].append(votes_cl.get_text())
                    else:
                        print("Votes are not available.")


                    if address is not None:
                        print("Address: ", address.get_text())
                        final_df['address'].append(address.get_text())
                    elif address_cl is not None:
                        print("Address: ", address_cl.get_text())
                        final_df['address'].append(address_cl.get_text())
                    else:
                        print("Address not availble")
                        
                        
                       
    return final_df

In [28]:
df_not_missing[:20]

,SVAAS_ID,firstName,lastName,name,baseCity,query
17,3,Divya,Kavuri,Apollo Clinic,Hyderabad,divya+kavuri+apollo clinic+hyderabad
18,4,Shivam,P,Shivam hospitals,Hyderabad,shivam+p+shivam hospitals+hyderabad
19,5,Ranjan,Dey,Ranjan skin clinics,Hyderabad,ranjan+dey+ranjan skin clinics+hyderabad
20,6,Soumya,Sarkar,Soumya Multi speciality clinic,Hyderabad,soumya+sarkar+soumya multi speciality clinic+h...
21,7,Rishanku,Goyal,RKG Eye Center,Hyderabad,rishanku+goyal+rkg eye center+hyderabad
22,8,Mamatha,R,Mamatha Clinics,Hyderabad,mamatha+r+mamatha clinics+hyderabad
24,150,Ravi,Pallapolu,SAI KEERTHI CHILDREN AND DENTAL CLINIC,Hyderabad,ravi+pallapolu+sai keerthi children and dental...
25,159,B,Ramesh,Aditya Medical Complex,Visakhapatnam,b +ramesh+aditya medical complex+visakhapatnam
26,160,V V,Satya Sekhar,Mamata Hospital,Visakhapatnam,v v +satya sekhar+mamata hospital+visakhapatnam
27,161,V,Ravi Chandra,SG Ortho Care,Visakhapatnam,v+ravi chandra+sg ortho care+visakhapatnam


In [29]:
final_df = extractReviews(df_not_missing)

  0%|                                                    | 0/20 [00:00<?, ?it/s]

Link searched for:  https://www.practo.com/hyderabad/doctor/divya-varikallu-general-practitioner


  5%|██▏                                         | 1/20 [00:04<01:28,  4.68s/it]

Name:  Dr. Divya Varikallu
Speciality:  General Practitioner
Rating:  5.0
Votes are not available.
Address:  61B, Sri Ram Nagar Colony, Botanical Garden Road, Landmark: Beside Lane of Burfighar Sweet Shop &  State Bank of India Lane, Hyderabad
Link searched for:  https://www.practo.com/hyderabad/clinic/shivam-clinic-kphb


 10%|████▍                                       | 2/20 [00:08<01:13,  4.10s/it]

...

Name:  Dr. Mohammed Akbar
Speciality:  Gastroenterologist
Rating not available.
Votes are not available.
Address:  Near KGH, Collector Offce Junction, Visakhapatnam
Link searched for:  https://www.practo.com/visakhapatnam/doctor/dr-sravani-sandhya-b-dermatologist-cosmetologist


 95%|████████████████████████████████████████▊  | 19/20 [00:52<00:02,  2.42s/it]

Name:  Dr. Sravani Sandhya B
Speciality:  Dermatologist
Rating:  84% 
Name:  (19 votes)
Address:  #50-49-43/4, TPT Colony, Seethammadhara., Landmark: Opposite Hindu Press., Visakhapatnam
Link searched for:  https://www.practo.com/visakhapatnam/doctor/dr-ch-s-padmaja-gynecologist-obstetrician
Name:  Dr. CH.S Padmaja
Speciality:  Gynecologist
Rating not available.
Votes are not available.
Address:  Seethammadara Jn,Visakhapatnam ,Landmark: Near Alluramalinga Statue., Visakhapatnam
Link searched for:  https://www.practo.com/visakhapatnam/doctor/a-soujanya-psychiatrist


100%|███████████████████████████████████████████| 20/20 [00:55<00:00,  2.78s/it]

Name:  Dr. A.Soujanya
Speciality:  Neuropsychiatrist
Rating:  96% 
Name:  (28 votes)
Address:  KGH Down, Landmark: Near Apollo Hospitals, Visakhapatnam


In [32]:
df_final = pd.DataFrame.from_dict(final_df, orient="index").T

practo: name, address, working days, phone, speciality

In [34]:
df_final

,SVAAS_ID,name,speciality,rating,address,votes
0,3,Dr. Divya Varikallu,General Practitioner,5.0,"61B, Sri Ram Nagar Colony, Botanical Garden Ro...",(20 patient stories)
1,3,Dr. Divya Varikallu,General Practitioner,5.0,"22-1-81/2, Shop Number 9&10, Sree Nivasam, Y J...",(184 votes)
2,4,Shivam Clinic,Orthopedics and Gynecology/Obstetrics Clinic,4.5,"#1-10-144, Temple Alwal Road, Alwal, Landmark:...",(303 votes)
3,5,Dr. Ranjan Kumar Dey,Urologist,94%,"Sainath Plaza, Maharanipeta, Landmark: Opp KGH...",(305 patient stories)
4,6,B P Poddar Hospital & Medical Research,General Physician,97%,"1st floor, MVV chambers, behind care-2 hospita...",(19 votes)
5,8,Dr. Mamatha,Ayurveda,5.0,"1st floor, MVV chambers, behind care-2 hospita...",(28 votes)
6,8,Dr. Mamatha,Neuropsychiatrist,5.0,"10-50-80, Waltair Main Road, Landmark: Opposit...",None
7,159,Dr. B Ramesh,General Physician,84%,"Near KGH, Collector Offce Junction, Visakhapatnam",None
8,162,Dr. Appala Naidu L S,Gastroenterologist,96%,"#50-49-43/4, TPT Colony, Seethammadhara., Land...",None
9,164,Dr. Reddy Sreenivasa Rao,Gastroenterology Clinic,None,"Seethammadara Jn,Visakhapatnam ,Landmark: Near...",None


In [ ]:
pd.merge(df, df_final, how='outer').to_csv("full_output_30_aug_2022.csv", index=False)

In [ ]:
pd.read_csv("full_output_30_aug_2022.csv")

In [ ]:
df

### Processing for NA

In [ ]:
df = pd.read_csv("full_output_30_aug_2022.csv")
df.head()

In [ ]:
df_na = df[df['rating'].isna() == True]

In [ ]:
df_na = df_na[df_na['firstName'].isna()!=True].drop('clinicName', axis=1)

In [ ]:
df_na.columns = ['firstName', 'lastName', 'name', 'baseCity', 'GoogleNRs', 'googleRating']

In [ ]:
df_na